In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import random
from urllib.request import urlopen
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [2]:
df = pd.read_csv('input_plot.csv')
th = pd.read_csv('movie.csv')
g = th['Director'][1142:].values.tolist()
l = th['Plot_reduction'][1142:].values.tolist()

In [3]:
# df.head()
# df = df[df['Genre']!='unknown']
# df = df[df['Release Year']>1950]
# df[["Title","Release Year","Genre",'Wiki Page']].head()
# # df.columns
# # df.shape
# df1 = df
# df.iloc[0]['Wiki Page']
def stem(ge,q):
    m = 12
    index = g.index(ge)
    if(index in [1,3,7]):
        m = 10
    w = l[index].split(',')
    s = q.split()
    n = int(len(s)/m)
    
    for j in range(1,n):
        s = s[:j*m] + [w[random.randint(0,len(w)-1)]] + s[j*m:]
    return ' '.join(s)

In [49]:
movie_name = input('Enter a movie name : ')
movie_name = movie_name.title()
movie = ""
for i in range(len(movie_name)):
    if movie_name[i] == ' ':
        movie += '+'
    else:
        movie += movie_name[i]
print(movie)
url = 'https://www.google.com/search?q='+movie
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
response = requests.get(url,headers = headers)
soup = BeautifulSoup(response.text, 'lxml')
gen = soup.find('div',{'data-attrid':'subtitle'}).find('span').text.lower()
gen = gen.split()[2]
print('genre : ',gen)
search_div = soup.find_all(class_='rc')
for result in search_div: 
    url = result.a.get('href') 
    if('wiki' in url):
        break
print(url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
url = "https://en.wikipedia.org"+soup.find('a',{'title':'Edit section: Plot'})['href']
print(url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
plot = soup.find('textarea').text
print(plot)

Enter a movie name : vaalu
Vaalu
genre :  comedy/romance
https://en.wikipedia.org/wiki/Vaalu
https://en.wikipedia.org/w/index.php?title=Vaalu&action=edit&section=1
==Plot==

Sharp aka Vaalu ([[Silambarasan]]) is a carefree, jobless youth leading a life without any worry in the world. He falls in love with Priya ([[Hansika Motwani]]) after seeing her in the rain in a bus stop. Priya has a maternal uncle Anbu ([[Aditya (actor)|Aditya]]), a powerful businessman whom she is to marry to fulfill her parents' wishes. The rest of the film deals with Sharp winning over Priya.



In [62]:
def func(plot,title):
    try:
        i = plot.index('==')
        plot = plot[i+len('=='):]
        i = plot.index('==')
        plot = plot[i+len('=='):]
    except:
        plot = plot
    while(1):
        i = plot.find('[[')
        if(i==-1):
            break
        j = plot.find(']]')
        plot = plot[:i] + plot[j+2:]
    while(1):
        i = plot.find('{{')
        if(i==-1):
            break
        j = plot.find('}}')
        plot = plot[:i] + plot[j+2:]
    while(1):
        i = plot.find('(')
        if(i==-1):
            break
        j = plot.find(')')
        plot = plot[:i] + plot[j+2:]
    p = ''

    for i in plot:
        if(i!='\n'):
            p += i
    plot = p
    stop_words = set(stopwords.words('english'))
    tagged_sentence = nltk.tag.pos_tag(plot.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    example_sentence = ' '.join(edited_sentence)
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~0123456789'''
    no_punct = ""
    for char in example_sentence:
       if char not in punctuations:
           no_punct = no_punct + char
       else:
            no_punct += ' '
    example_sentence = no_punct
    tagged_sentence = nltk.tag.pos_tag(example_sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag not in ['NNP','NNPS','NNS','NN']]
    example_sentence = ' '.join(edited_sentence)
    example_sentence = example_sentence.lower()
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(example_sentence) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filt = [ps.stem(filtered_sentence[i]) for i in range(len(filtered_sentence))]
    filt = ' '.join(filt)
    for i in g:
        if i in gen:
            filt = stem(i,filt)
    df1 = pd.DataFrame([title,gen,filt],['title','genre','Plot_reduction'],columns = [df.shape[0]]).T
    return df1

a = func(plot,movie_name)

In [63]:
a['Plot_reduction'].values

array(['sharp carefre lead without fall see matern power marri fulfil wish win'],
      dtype=object)

In [64]:
pd.concat([df,a]).to_csv('input_plot.csv',index = False)

,title,genre,Plot_reduction
0,ratchasan,thriller,open old found brutal murder wrap aspir want m...
1,96,romance,visit high overcom arrang catch hesitantli lov...
2,santhosh subramaniyam,comedy/romance,rich manag live excess dote upon younger love ...
3,kutram 23,thriller,start confess murder abduct celebr anniversari...
4,vtv,romance,fall meet white got know older tri interact af...
5,thozha,drama,wealthi own leav ensur legal convey interest m...
6,silence,thriller,near mysteri murder crucifi take remain unsold...
7,anandham,drama,own provision eldest among four live togeth al...
8,zodiac,thriller,featur unknown surviv one later written call t...
9,the silence of the lambs,thriller,pull assign former incarcer cannibalist serial...


In [ ]:
a